# API Testing Notebook

This notebook provides comprehensive testing for the Customer Matching POC API endpoints.

## Available Endpoints:
- **Health**: `/api/v1/health` - System health check
- **Customers**: `/api/v1/customers` - Customer management
- **Matching**: `/api/v1/matching` - Customer matching operations

## Setup
Make sure the API server is running on `http://localhost:8000`

In [ ]:
# Import required libraries
import requests
import json
import time
from datetime import datetime
import pandas as pd
from typing import Dict, List, Any

# Configuration
BASE_URL = "http://localhost:8000"
API_BASE = f"{BASE_URL}/api/v1"

print(f"API Base URL: {API_BASE}")
print(f"API Documentation: {BASE_URL}/docs")
print(f"ReDoc Documentation: {BASE_URL}/redoc")

## 1. Health Check Testing

In [ ]:
# Test health endpoint
def test_health_check():
    """Test the health check endpoint"""
    try:
        response = requests.get(f"{API_BASE}/health")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            health_data = response.json()
            print("\nHealth Check Results:")
            print(f"Status: {health_data.get('status')}")
            print(f"Version: {health_data.get('version')}")
            print(f"Database Connected: {health_data.get('database_connected')}")
            print(f"OpenAI Connected: {health_data.get('openai_connected')}")
            print(f"Timestamp: {health_data.get('timestamp')}")
            
            return health_data
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# Run health check
health_result = test_health_check()

## 2. Customer Management Testing

In [ ]:
# Test customer creation
def create_test_customer():
    """Create a test customer"""
    customer_data = {
        "company_name": "TechCorp Solutions Inc.",
        "contact_name": "John Smith",
        "email": "john.smith@techcorp.com",
        "phone": "+1-555-0123",
        "address": "123 Tech Street",
        "city": "San Francisco",
        "state": "CA",
        "country": "USA",
        "postal_code": "94105",
        "industry": "Technology",
        "company_size": "Medium",
        "annual_revenue": "$10M-$50M",
        "website": "https://techcorp.com",
        "notes": "Leading technology solutions provider"
    }
    
    try:
        response = requests.post(
            f"{API_BASE}/customers",
            json=customer_data,
            headers={"Content-Type": "application/json"}
        )
        
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            customer = response.json()
            print(f"\nCreated Customer:")
            print(f"Customer ID: {customer.get('customer_id')}")
            print(f"Company: {customer.get('company_name')}")
            print(f"Contact: {customer.get('contact_name')}")
            print(f"Email: {customer.get('email')}")
            return customer
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# Create test customer
test_customer = create_test_customer()

In [ ]:
# Test listing customers
def list_customers(limit: int = 10):
    """List all customers"""
    try:
        response = requests.get(f"{API_BASE}/customers?limit={limit}")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            customers = response.json()
            print(f"\nFound {len(customers)} customers:")
            
            # Create a DataFrame for better display
            if customers:
                df = pd.DataFrame(customers)
                display(df[['customer_id', 'company_name', 'contact_name', 'email', 'city', 'country']])
            
            return customers
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# List customers
customers = list_customers()

## 3. Incoming Customer Testing

In [ ]:
# Test incoming customer creation
def create_incoming_customer():
    """Create an incoming customer for matching"""
    incoming_data = {
        "company_name": "TechCorp Solutions",
        "contact_name": "John Smith",
        "email": "john.smith@techcorp.com",
        "phone": "+1-555-0123",
        "address": "123 Tech Street",
        "city": "San Francisco",
        "state": "CA",
        "country": "USA",
        "postal_code": "94105",
        "industry": "Technology",
        "company_size": "Medium",
        "annual_revenue": "$10M-$50M",
        "website": "https://techcorp.com",
        "notes": "Looking for technology solutions",
        "source": "Website Form",
        "priority": "High"
    }
    
    try:
        response = requests.post(
            f"{API_BASE}/customers/incoming",
            json=incoming_data,
            headers={"Content-Type": "application/json"}
        )
        
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            incoming = response.json()
            print(f"\nCreated Incoming Customer:")
            print(f"Request ID: {incoming.get('request_id')}")
            print(f"Company: {incoming.get('company_name')}")
            print(f"Contact: {incoming.get('contact_name')}")
            print(f"Status: {incoming.get('processing_status')}")
            return incoming
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# Create incoming customer
incoming_customer = create_incoming_customer()

## 4. Customer Search Testing

In [ ]:
# Test customer search functionality
def search_similar_customers(query_text: str, threshold: float = 0.7, max_results: int = 5):
    """Search for similar customers using text query"""
    search_data = {
        "query_text": query_text,
        "similarity_threshold": threshold,
        "max_results": max_results
    }
    
    try:
        response = requests.post(
            f"{API_BASE}/customers/search",
            json=search_data,
            headers={"Content-Type": "application/json"}
        )
        
        print(f"Status Code: {response.status_code}")
        print(f"Search Query: '{query_text}'")
        print(f"Similarity Threshold: {threshold}")
        print(f"Max Results: {max_results}")
        
        if response.status_code == 200:
            results = response.json()
            print(f"\nFound {len(results)} similar customers:")
            
            if results:
                df = pd.DataFrame(results)
                display(df[['customer_id', 'company_name', 'contact_name', 'email', 'similarity_score']])
            
            return results
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# Test different search queries
search_queries = [
    "technology company in San Francisco",
    "software solutions provider",
    "medium sized tech business",
    "John Smith contact information"
]

for query in search_queries:
    print(f"\n{'='*60}")
    search_results = search_similar_customers(query)
    print(f"{'='*60}\n")

## 5. Customer Matching Testing

In [ ]:
# Test customer matching (standard)
def test_customer_matching(request_id: int):
    """Test customer matching for a specific incoming customer"""
    try:
        print(f"Processing matching for request_id: {request_id}")
        start_time = time.time()
        
        response = requests.post(f"{API_BASE}/matching/{request_id}")
        
        processing_time = (time.time() - start_time) * 1000
        print(f"API Response Time: {processing_time:.2f}ms")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"\nMatching Results:")
            print(f"Incoming Customer: {result.get('incoming_customer', {}).get('company_name')}")
            print(f"Total Matches: {result.get('total_matches')}")
            print(f"Processing Time: {result.get('processing_time_ms')}ms")
            
            # Display matches
            matches = result.get('matches', [])
            if matches:
                print(f"\nTop Matches:")
                df = pd.DataFrame(matches)
                display(df[['matched_customer_id', 'matched_company_name', 'similarity_score', 'match_type']])
            
            return result
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# Test matching if we have an incoming customer
if incoming_customer and 'request_id' in incoming_customer:
    request_id = incoming_customer['request_id']
    matching_result = test_customer_matching(request_id)
else:
    print("No incoming customer available for matching test")

In [ ]:
# Test hybrid matching
def test_hybrid_matching(request_id: int):
    """Test hybrid customer matching"""
    try:
        print(f"Processing hybrid matching for request_id: {request_id}")
        start_time = time.time()
        
        response = requests.post(f"{API_BASE}/matching/hybrid/{request_id}")
        
        processing_time = (time.time() - start_time) * 1000
        print(f"API Response Time: {processing_time:.2f}ms")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"\nHybrid Matching Results:")
            print(f"Incoming Customer: {result.get('incoming_customer', {}).get('company_name')}")
            print(f"Total Matches: {result.get('total_matches')}")
            print(f"Processing Time: {result.get('processing_time_ms')}ms")
            
            # Display matches
            matches = result.get('matches', [])
            if matches:
                print(f"\nTop Matches:")
                df = pd.DataFrame(matches)
                display(df[['matched_customer_id', 'matched_company_name', 'similarity_score', 'match_type']])
            
            return result
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# Test hybrid matching if we have an incoming customer
if incoming_customer and 'request_id' in incoming_customer:
    request_id = incoming_customer['request_id']
    hybrid_result = test_hybrid_matching(request_id)
else:
    print("No incoming customer available for hybrid matching test")

## 6. Summary and Cleanup

In [ ]:
# Summary function
def generate_test_summary():
    """Generate a summary of all tests performed"""
    print("API Testing Summary")
    print("=" * 50)
    print(f"Test Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"API Base URL: {API_BASE}")
    print(f"Documentation: {BASE_URL}/docs")
    
    # Check if we have test data
    if 'health_result' in locals() and health_result:
        print(f"\n✓ Health Check: PASSED")
        print(f"  - Database: {'✓' if health_result.get('database_connected') else '✗'}")
        print(f"  - OpenAI: {'✓' if health_result.get('openai_connected') else '✗'}")
    else:
        print(f"\n✗ Health Check: FAILED")
    
    if 'test_customer' in locals() and test_customer:
        print(f"✓ Customer Creation: PASSED (ID: {test_customer.get('customer_id')})")
    else:
        print(f"✗ Customer Creation: FAILED")
    
    if 'incoming_customer' in locals() and incoming_customer:
        print(f"✓ Incoming Customer Creation: PASSED (ID: {incoming_customer.get('request_id')})")
    else:
        print(f"✗ Incoming Customer Creation: FAILED")
    
    if 'matching_result' in locals() and matching_result:
        print(f"✓ Customer Matching: PASSED ({matching_result.get('total_matches')} matches)")
    else:
        print(f"✗ Customer Matching: FAILED")
    
    print("\nTest completed successfully!")

# Generate summary
generate_test_summary()